In [1]:
from time import time
import numpy as np

from sklearn import svm
from sklearn.linear_model import SGDClassifier

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn import datasets

In [2]:
import sys
import os
# sys.path.append("../include/")
basePath = os.getcwd()
basePath = os.path.split(basePath)[0]
basePath = os.path.split(basePath)[0]
sys.path.append(os.path.join(basePath, "include"))
print(basePath)

D:\Gnutz\Documents\Uni\Semester6\MAL\IKT-MAL


In [3]:
from libitmal import dataloaders as itmaldataloaders

#%%
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

data_csv_path =  basePath + "/Slutprojekt/DatafordelingPlot/spotifyDBData_preprocessed_minmaxscaled.csv"
data_csv_path_onehot = basePath + "/Slutprojekt/DatafordelingPlot/spotifyDBData_preprocessed_minmaxscaled_onehotenc-all.csv"
data_csv_path_onehot_mode_tsig = basePath + "/Slutprojekt/DatafordelingPlot/spotifyDBData_preprocessed_minmaxscaled_onehotenc-mode-tsig.csv"
data_csv_path_onehot_mode_tsig_stdscaled = basePath + "/Slutprojekt/DatafordelingPlot/spotifyDBData_preprocessed_stdscaled_onehotenc-mode-tsig.csv"

def loadPreprocessed():
    return pd.read_csv(data_csv_path, sep=',', header=0)

def loadPreprocessedOnehotenc():
    return pd.read_csv(data_csv_path_onehot, sep=',', header=0)

def loadPreprocessedModeTsigOnehotenc():
    return pd.read_csv(data_csv_path_onehot_mode_tsig, sep=',', header=0)

def loadPreprocessedModeTsigOnehotencStdScaled():
    return pd.read_csv(data_csv_path_onehot_mode_tsig_stdscaled, sep=',', header=0)

all_genres = ['Alternative', 'Anime', 'Blues', 'Children’s Music', 'Classical', 'Comedy',
 'Country', 'Dance', 'Electronic', 'Folk', 'Hip-Hop', 'Indie', 'Jazz', 'Movie',
 'Opera', 'Pop', 'R&B', 'Rap', 'Reggae', 'Reggaeton', 'Rock', 'Ska', 'Soul',
 'Soundtrack', 'World']

regular_input_features = [
    "popularity", "acousticness", "danceability",
    "energy", "instrumentalness",  "liveness", "loudness", "speechiness",
    "tempo", "valence"
]

onehot_input_features = [
    "mode_major", "mode_minor",
    "time_signature_3-4", "time_signature_4-4", "time_signature_5-4"
]

def getSpotifyBinarizedXY(remove_cols=None):
    data_filtered = loadPreprocessed()
    if (remove_cols is not None):
        for col in remove_cols:
            data_filtered = data_filtered[data_filtered.genre != col]
    X_df = data_filtered[regular_input_features]
    lb = LabelBinarizer()
    X = np.array(X_df)
    y = lb.fit_transform(np.array(data_filtered['genre']))

    print("Binarized Label to the following classes")
    print(lb.classes_)
    print(lb.classes_.shape[0])

    return X, y

def getSpotifyIntLabeledXY():
    spotifyDBData = loadPreprocessed()
    X = np.array(spotifyDBData[regular_input_features])

    label_encoder = LabelEncoder() 
    y = label_encoder.fit_transform(spotifyDBData['genre']) 

    return X, y

def getSpotifyOneHotEncXY():
    spotifyDBData = loadPreprocessedModeTsigOnehotenc()
    input_features = regular_input_features + onehot_input_features
    X = np.array(spotifyDBData[input_features])

    y = LabelEncoder().fit_transform(spotifyDBData['genre']) 
    return X, y

def getSpotifyOneHotEncStdScaledXY():
    spotifyDBData = loadPreprocessedModeTsigOnehotencStdScaled()
    input_features = regular_input_features + onehot_input_features
    X = np.array(spotifyDBData[input_features])

    y = LabelEncoder().fit_transform(spotifyDBData['genre']) 
    return X, y

#%%
currmode="N/A" # GLOBAL var!

def SearchReport(model): 
    
    def GetBestModelCTOR(model, best_params):
        def GetParams(best_params):
            r=""          
            for key in sorted(best_params):
                value = best_params[key]
                t = "'" if str(type(value))=="<class 'str'>" else ""
                if len(r)>0:
                    r += ','
                r += f'{key}={t}{value}{t}'  
            return r            
        try:
            p = GetParams(best_params)
            return type(model).__name__ + '(' + p + ')' 
        except:
            return "N/A(1)"
        
    print("\nBest model set found on train set:")
    print()
    print(f"\tbest parameters={model.best_params_}")
    print(f"\tbest '{model.scoring}' score={model.best_score_}")
    print(f"\tbest index={model.best_index_}")
    print()
    print(f"Best estimator CTOR:")
    print(f"\t{model.best_estimator_}")
    print()
    try:
        print(f"Grid scores ('{model.scoring}') on development set:")
        means = model.cv_results_['mean_test_score']
        stds  = model.cv_results_['std_test_score']
        i=0
        for mean, std, params in zip(means, stds, model.cv_results_['params']):
            print("\t[%2d]: %0.3f (+/-%0.03f) for %r" % (i, mean, std * 2, params))
            i += 1
    except:
        print("WARNING: the random search do not provide means/stds")
    
    global currmode                
    assert "f1_micro"==str(model.scoring), f"come on, we need to fix the scoring to be able to compare model-fits! Your scoreing={str(model.scoring)}...remember to add scoring='f1_micro' to the search"   
    return f"best: dat={currmode}, score={model.best_score_:0.5f}, model={GetBestModelCTOR(model.estimator,model.best_params_)}", model.best_estimator_ 

def ClassificationReport(model, X_test, y_test, target_names=None):
    assert X_test.shape[0]==y_test.shape[0]
    print("\nDetailed classification report:")
    print("\tThe model is trained on the full development set.")
    print("\tThe scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, model.predict(X_test)                 
    print(classification_report(y_true, y_pred, target_names))
    print()
    
def FullReport(model, X_test, y_test, t):
    print(f"SEARCH TIME: {t:0.2f} sec")
    beststr, bestmodel = SearchReport(model)
    ClassificationReport(model, X_test, y_test)    
    print(f"CTOR for best model: {bestmodel}\n")
    print(f"{beststr}\n")
    return beststr, bestmodel
    
def LoadAndSetupData(mode, test_size=0.3):
    assert test_size>=0.0 and test_size<=1.0
    
    def ShapeToString(Z):
        n = Z.ndim
        s = "("
        for i in range(n):
            s += f"{Z.shape[i]:5d}"
            if i+1!=n:
                s += ";"
        return s+")"

    global currmode
    currmode=mode
    print(f"DATA: {currmode}..")
    
    if mode=='moon':
        X, y = itmaldataloaders.MOON_GetDataSet(n_samples=5000, noise=0.2)
        itmaldataloaders.MOON_Plot(X, y)
    elif mode=='mnist':
        X, y = itmaldataloaders.MNIST_GetDataSet(fetchmode=False)
        if X.ndim==3:
            X=np.reshape(X, (X.shape[0], -1))
    elif mode=='iris':
        X, y = itmaldataloaders.IRIS_GetDataSet()
    elif mode=='spotify_binarized':
        #remove_cols = ['Alternative', 'Anime', 'Blues', 'Children’s Music', 'Classical', 'Comedy',
            # 'Country', 'Dance', 'Electronic', 'Folk', 'Hip-Hop', 'Indie', 'Jazz', 'Movie',
            # 'Opera', 'Pop', 'R&B', 'Reggae', 'Reggaeton']
        remove_cols = None
        X, y = getSpotifyBinarizedXY(remove_cols)
    elif mode=='spotify_intlabels':
        X, y = getSpotifyIntLabeledXY()
    elif mode=='spotify_onehotin_labelout':
        X, y = getSpotifyOneHotEncXY()
    elif mode=='spotify_onehotin_labelout_std':
        X, y = getSpotifyOneHotEncStdScaledXY()
    else:
        raise ValueError(f"could not load data for that particular mode='{mode}'")
        
    print(f'  org. data:  X.shape      ={ShapeToString(X)}, y.shape      ={ShapeToString(y)}')

    assert X.ndim==2
    assert X.shape[0]==y.shape[0]
    # assert y.ndim==1 or (y.ndim==2 and y.shape[1]==0)    
    
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=0, shuffle=True
    )
    
    print(f'  train data: X_train.shape={ShapeToString(X_train)}, y_train.shape={ShapeToString(y_train)}')
    print(f'  test data:  X_test.shape ={ShapeToString(X_test)}, y_test.shape ={ShapeToString(y_test)}')
    print()
    
    return X_train, X_test, y_train, y_test

print('OK')


Using TensorFlow backend.


OK


In [5]:
#%% -----------------------------------
#   Neural Network
# -------------------------------------

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn import datasets

import numpy as np
from time import time

np.random.seed(42)

X_train, X_test, y_train, y_test = LoadAndSetupData('spotify_onehotin_labelout')

y_train_binary = to_categorical(y_train)
y_test_binary  = to_categorical(y_test)

assert y_train_binary.ndim==2
assert y_test_binary.ndim ==2

DATA: spotify_onehotin_labelout..
  org. data:  X.shape      =(222593;   15), y.shape      =(222593)
  train data: X_train.shape=(155815;   15), y_train.shape=(155815)
  test data:  X_test.shape =(66778;   15), y_test.shape =(66778)



In [6]:
from keras import backend as K

def recall_m(y, y_pred,**kwargs):
        true_positives = K.sum(K.round(K.clip(y * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y, y_pred, **kwargs):
        true_positives = K.sum(K.round(K.clip(y * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y, y_pred, **kwargs):
    precision = precision_m(y, y_pred)
    recall = recall_m(y, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

# compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc',f1_m,precision_m, recall_m])

# # fit the model
# history = model.fit(Xtrain, ytrain, validation_split=0.3, epochs=10, verbose=0)

# # evaluate the model
# loss, accuracy, f1_score, precision, recall = model.evaluate(Xtest, ytest, verbose=0)

#%%

# Build Keras model 
model = Sequential()
model.add(Dense(input_dim=15, units=20, activation="tanh", kernel_initializer="normal"))
model.add(Dense(units=25, activation="softmax"))

#optimizer = SGD(lr=0.1)
optimizer = Adam(lr=0.1)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer, 
              metrics=['acc', f1_m, precision_m, recall_m])

# Train
VERBOSE     = 10
EPOCHS      = 35

start = time()
history = model.fit(X_train, y_train_binary, 
    validation_data=(X_test, y_test_binary), 
    epochs=EPOCHS, verbose=VERBOSE)
t = time()-start

print(f"OK, training time={t:0.1f}")


#%%
# score = model.evaluate(X_test, y_test_binary, verbose=0)

# print(f"Training time: {t:0.1f} sec")
# print({score[0]}) # loss is score 0 by definition?
# print({score[1]})
# print(f"All scores in history: {score}")

loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test_binary, verbose=10)
print("loss: " + str(loss))
print("accuracy: " + str(accuracy))
print("f1_score: " + str(f1_score))
print("precision: " + str(precision))
print("recall: " + str(recall))

# from sklearn.metrics import classification_report

# y_pred = model.predict(x_test, batch_size=64, verbose=1)
# y_pred_bool = np.argmax(y_pred, axis=1)

# print(classification_report(y_test, y_pred_bool))







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 155815 samples, validate on 66778 samples
Epoch 1/35
Epoch 2/35
Epoch 3/35
Epoch 4/35
Epoch 5/35
Epoch 6/35
Epoch 7/35
Epoch 8/35
Epoch 9/35
Epoch 10/35
Epoch 11/35
Epoch 12/35
Epoch 13/35
Epoch 14/35
Epoch 15/35
Epoch 16/35
Epoch 17/35
Epoch 18/35
Epoch 19/35
Epoch 20/35
Epoch 21/35
Epoch 22/35
Epoch 23/35
Epoch 24/35
Epoch 25/35
Epoch 26/35
Epoch 27/35
Epoch 28/35
Epoch 29/35
Epoch 30/35
Epoch 31/35
Epoch 32/35
Epoch 33/35
Epoch 34/35
Epoch 35/35
OK, training time=353.3
loss: 2.4539380864229603
accuracy: 0.3046512324440536
f1_score: 0.27087024911604834
precision: 0.46797122095170596
recall: 0.19316241876178541


In [13]:
y_predicts = model.predict(X_train)
print(y_predicts[0])

[3.1942517e-07 1.3364031e-03 4.3438195e-06 2.4377313e-04 8.3662551e-03
 3.2706046e-04 1.3754796e-07 2.4630648e-08 8.2499349e-07 5.8108355e-07
 2.5565783e-09 2.7673300e-07 6.5506324e-06 5.0738966e-03 9.8271549e-01
 1.0238780e-07 1.2231132e-07 7.5731116e-08 1.7533310e-07 1.4775224e-08
 3.7394869e-07 1.5630436e-05 1.5736983e-06 9.4003708e-04 9.6594944e-04]


In [8]:
# GridSearch
# Use scikit-learn to grid search the number of neurons
from sklearn.model_selection import GridSearchCV
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import make_scorer

In [7]:
# # GridSearch
# # Use scikit-learn to grid search the number of neurons
# from sklearn.model_selection import GridSearchCV
# # from keras.models import Sequential
# # from keras.layers import Dense
# # from keras.layers import Dropout
# from keras.wrappers.scikit_learn import KerasClassifier
# from keras.constraints import maxnorm
# from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
# from sklearn.metrics import make_scorer

# # scoring = {'acc': make_scorer(accuracy_score), 'f1': make_scorer(f1_score),
# #                              'precision': make_scorer(precision_score),'recall' : make_scorer(recall_score)}
# # scoring = {'acc': make_scorer(accuracy_score), 'f1': f1_m,
# #                              'precision': precision_m,'recall' : recall_m}
# # scoring = {'f1': make_scorer(f1_m), 'precision': make_scorer(precision_m),'recall' : make_scorer(recall_m)}

# def createModel(neurons=1):
#     # Build Keras model 
#     model = Sequential()
#     model.add(Dense(input_dim=15, units=neurons, activation="tanh", kernel_initializer="normal"))
#     model.add(Dense(units=25, activation="softmax"))

#     #optimizer = SGD(lr=0.1)
#     optimizer = Adam(lr=0.1)
#     model.compile(loss='categorical_crossentropy', 
#                   optimizer=optimizer, 
#                   metrics=['accuracy'])
#     return model

# model = KerasClassifier(build_fn=createModel, epochs=100, batch_size=10, verbose=10)
# # define the grid search parameters
# neurons = [1, 5, 10, 15, 20, 25, 30]
# param_grid = dict(neurons=neurons)
# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=5, verbose=10)
# grid.fit(X_train, y_train_binary, verbose=10)
# # # summarize results
# # print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# # # means = grid_result.cv_results_['mean_test_score']
# # # stds = grid_result.cv_results_['std_test_score']
# # # params = grid_result.cv_results_['params']
# # # for mean, stdev, param in zip(means, stds, params):
# # #     print("%f (%f) with: %r" % (mean, stdev, param))

In [38]:
scorer = make_scorer(f1_score, average='weighted')

def create_model():
    # Build Keras model 
    model = Sequential()
    model.add(Dense(input_dim=15, units=20, activation="tanh", kernel_initializer="normal"))
    model.add(Dense(units=25, activation="softmax"))

    #optimizer = SGD(lr=0.1)
    optimizer = Adam(lr=0.1)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=optimizer, 
                  metrics=['accuracy'])
    return model


model = KerasClassifier(build_fn=create_model, verbose=10)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [15, 30, 45, 60, 75, 90, 105]
# batch_size = [10]
# epochs = [2]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, scoring=scorer, verbose=10)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# f1 = grid_result.cv_results_['f1']
# prec = grid_result.cv_results_['prec']
# rec = grid_result.cv_results_['rec']
# params = grid_result.cv_results_['params']
# for f1, prec,  rec, param in zip(f1, prec, rec, params):
#     print("%f (%f) (%f) with: %r" % (f1, prec, rec, params))

Fitting 3 folds for each of 42 candidates, totalling 126 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
exception calling callback for <Future at 0x1d50b9274a8 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "D:\Gnutz\Anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "D:\Gnutz\Anaconda3\lib\site-packages\joblib\parallel.py", line 309, in __call__
    self.parallel.dispatch_next()
  File "D:\Gnutz\Anaconda3\lib\site-packages\joblib\parallel.py", line 731, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "D:\Gnutz\Anaconda3\lib\site-packages\joblib\parallel.py", line 759, in dispatch_one_batch
    self._dispatch(tasks)
  File "D:\Gnutz\Anaconda3\lib\site-packages\joblib\parallel.py", line 716, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "D:\Gnutz\Anaconda3\lib\site-packages\joblib\_parallel_backends.py", line 510, in apply_async
    future = s

TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.

In [37]:
grid_result.best_score_

0.2325567335842111

In [9]:
print(X_train.shape)
print(y_train.shape, y_train)
print(y_train_binary.shape)
print(y_train_bina
      ry[20])

(155815, 15)
(155815,) [14  3 19 ... 13  9 24]
(155815, 25)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0.]
